## 1. Connection to the database

In [1]:
# Import the necessary libraries
import csv
import sqlite3

# Connect to the database
connexion = sqlite3.connect("db/vivino.db")
cursor = connexion.cursor()

In [2]:
# Query the name of all tables of the database
cursor.execute("""
    SELECT name
    FROM sqlite_master
    WHERE type='table';
""")
for name, in cursor.fetchall():
    print(name)

countries
grapes
wineries
flavor_groups
keywords
regions
most_used_grapes_per_country
toplists
wines
vintages
keywords_wine
vintage_toplists_rankings


In [3]:
# A query to get the three most used grapes in the world
cursor.execute("""
    SELECT
        g.name AS grape_name,
        COUNT(DISTINCT m.country_code) AS countries_count
    FROM most_used_grapes_per_country AS m
    LEFT JOIN grapes AS g
        ON m.grape_id = g.id
    GROUP BY g.name
    ORDER BY countries_count DESC
    LIMIT 3
""")

cursor.fetchall()

[('Cabernet Sauvignon', 12), ('Merlot', 11), ('Chardonnay', 6)]

In [4]:
cursor.execute("""
        SELECT
               w.name AS wine_name,
               s.grape_name AS grape_name,
               w.ratings_average AS wine_ratings_average,
               w.ratings_count AS wine_ratings_count
        FROM wines AS w
        LEFT JOIN regions AS r ON w.region_id = r.id
        LEFT JOIN countries AS c ON r.country_code = c.code
        INNER JOIN
            -- A subquery to get the three most used grapes in the world
            (SELECT
                g.name AS grape_name,
                COUNT(DISTINCT c.name) AS countries_count,
                c.code AS country_code
            FROM most_used_grapes_per_country AS m
            LEFT JOIN grapes AS g
                ON m.grape_id = g.id
            LEFT JOIN countries AS c
                ON m.country_code = c.code
            GROUP BY g.name
            ORDER BY countries_count DESC
            LIMIT 3) AS s
                ON c.code = s.country_code
        ORDER BY wine_ratings_average DESC, wine_ratings_count DESC;
""")

data_01 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_01.append(list(item))

In [5]:
with open('result_01.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_01)

In [9]:
# Query the wine name containing one of the following string:
# 'Cabernet Sauvignon', 'Merlot', and 'Chardonnay'
cursor.execute("""
       SELECT
              name AS wine_name,
              CASE WHEN name LIKE '%Cabernet Sauvignon%' THEN 'Cabernet Sauvignon'
                   WHEN name LIKE '%Merlot%' THEN 'Merlot'
                   WHEN name LIKE '%Chardonnay%' THEN 'Chardonnay'
                   END AS grape_name,
              ratings_average,
              ratings_count
       FROM wines
       WHERE name LIKE '%Cabernet Sauvignon%'
              OR name LIKE '%Merlot%'
              OR name LIKE '%Chardonnay%'
       ORDER BY ratings_average DESC, ratings_count DESC;
""")

data_02 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_02.append(list(item))

OperationalError: unrecognized token: "@"

In [7]:
with open('result_02.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_02)

In [14]:
# Query the wine name containing the string 'Cabernet Sauvignon'
cursor.execute("""
    SELECT
        w.name AS wine_name,
        v.year,
        w.ratings_average,
        w.ratings_count,
        (v.price_euros/v.bottle_volume_ml)*1000 AS liter_price_euros,
        c.name AS country
    FROM wines AS w
    INNER JOIN vintages AS v ON w.id = v.wine_id
    LEFT JOIN regions AS r ON w.region_id = r.id
    LEFT JOIN countries AS c ON r.country_code = c.code
    WHERE w.name LIKE '%Cabernet Sauvignon%'
    ORDER BY w.ratings_average DESC, w.ratings_count DESC;
""")

data_03 = [[item[0] for item in cursor.description]]

for item in cursor.fetchall():
    data_03.append(list(item))

In [15]:
with open('result_03.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data_03)

In [ ]:
def query_to_csv(query: str, csv_file_name: str):
    """
    Args:

    Returns:
    """
    connexion = sqlite3.connect("db/vivino.db")
    cursor = connexion.cursor()
    cursor.execute(query)
    data = [[item[0] for item in cursor.description]]
    for item in cursor.fetchall():
        data.append(list(item))
    
    with open(csv_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)
    
    connexion.close()